<img src='https://pennstatelaw.psu.edu/sites/default/files/styles/content_header_image/public/M%20and%20A%20center%201.jpg?itok=nxlHzngRhttps://pennstatelaw.psu.edu/sites/default/files/styles/content_header_image/public/M%20and%20A%20center%201.jpg?itok=nxlHzngR' height="100%" width="3500">

# Projeto Quant.dev

Olá! Seja bem vindo ao projeto de Quant do FEA.dev!

O que você irá fazer:

  "*Você é um sócio e analista quantitativo do nosso Fundo. Seu objetivo é apresentar sua estratégia de investimento e convencer os outros sócios a colocá-la no portfólio. Você tem boa parte do seu patrimônio pessoal investido nesse Fundo.*"

### Você decidiu implementar a seguinte estratégia:

A sua hipótese é que as ações mais negociadas no mês, ou seja, de maior volume, irão bem no mês seguinte. A sua regra de investimento será comprar, todo final de mês, as 10 ações com o maior volume (até o momento da compra) do indice IBX. Você tem um montante inicial de R$100.000 para investir.

Comece extraindo os dados de preços e volume, tome cuidado que nem todas as fontes te dão essas informações. 

A composição do IBX a cada ano lhe foi dada no arquivo excel. O período de sua estratégia deve contemplar todo o período do mesmo arquivo.

Depois que sua estratégia estiver rodando sem erros, faça uma **análise minuciosa** da estratégia (a maior parte da sua avaliação de desempenho se dará nessa parte). Para isso, use métricas diversas (retorno, volatilidade, drawdown, etc.) plotando e comparando-as. Também faça variações dessa estratégia (comprar as 5 ações com o melhor Sharpe, 10 ações com o melhor Sharpe, 15 ações com o melhor Sharpe, etc) e veja qual é a melhor.

Por fim, deixe mais do que claro se:

1. Está confiante para indicar essa ideia para o portfólio. ou  
2. Não chegou a um resultado satisfatório. ou  
3. Acha que precisa de mais estudos para se sentir confiante.

Obs: Não há problema se algumas ações derem erro na hora da extração, siga em frente. Mas se muitas estiverem dando erro, melhor revisar seu código...

### Materiais de apoio:
1. [Coursera: Introduction to Portfolio Construction and Analysis with Python (link)](https://www.coursera.org/programs/universidade-de-sao-paulo-br-on-coursera-mvxtw?currentTab=MY_COURSES&productId=qeyXSV_uEeiNIQqyU-T6bA&productType=course&showMiniModal=true) 

    - Semanas 1 e 2 

2. [Playlist de Finanças Quantitativas do FEA.dev (link)](https://www.youtube.com/playlist?list=PLiJqHBsOKi2cabSus1hgdSNAHeU0md6_D)

3. Extração de dados do Banco Central de forma simplificada https://medium.com/fea-dev/extra%C3%A7%C3%A3o-de-dados-do-banco-central-de-forma-simplificada-7b946069c03e
3. Extras (**Não são necessários**, apenas para motivá-los)
    - What is Quant Finance: https://www.youtube.com/watch?v=l7imVaK0MIQ
    - What is a Quant Trader? | Systematic Investing | What is a Quant Hedge Fund? | Trading Ideas: https://www.youtube.com/watch?v=EBZVtDZw9ug
    - The Story of James Simons - Renaissance Technologies & Medallion Fund: https://www.youtube.com/watch?v=xkbdZb0UPac&pp=ugMICgJwdBABGAE%3D
    - Quants Os Alquimistas de Wall Street:  https://www.youtube.com/watch?v=ed2FWNWwE3I
    - Nossa planilha de recomendações (blogs, sites, canais do youtube): https://docs.google.com/spreadsheets/d/12YnAl8Tmd5fTDUL__3dEKsnQ1G28yKNaOLRt3Hq3aZs/edit#gid=0
    - What is Quantitative Finance?: https://medium.com/swlh/what-is-quantitative-finance-193ee8788ed4#:~:text=It%20is%20a%20field%20of,trading%2C%20and%20predicting%20market%20moves
    

In [7]:
# librarys
# api key: WKILRJ27Z51OABEO
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('ggplot')

In [8]:
prices = pd.read_csv('data/prices.csv', index_col=0)

In [9]:
volume = pd.read_csv('data/volume.csv', index_col=0)

In [10]:
# returos percentuais
returns = prices.pct_change()
returns.index = pd.to_datetime(returns.index)

# retorno logarítimico
log_returns = np.log(prices) - np.log(prices).shift(1)

In [11]:
# retorno cumulativo
cum_return = (returns + 1).prod() - 1

# retorno cumulativo log
cum_return_log = np.exp(np.sum(log_returns)) - 1

# print
#print(f'retorno cumulativo pct: {cum_return}\n retorno cumulativo log:{cum_return_log}')
if any(round(cum_return, 4) == round(cum_return_log, 4)):
    print('São iguais, excelente')
else:
    print('Tem que ver aí...')

São iguais, excelente


In [12]:
# retornos anualizados
annualized_returns = cum_return ** (12/returns.shape[0]) - 1

# sharpe ratio
rf = 0.03
sharpe_ratio = (annualized_returns - rf)/annualized_returns

# Estratégia dos volumes

In [179]:
def get_portfolio_returns(df_volume, df_returns, n_largest=10):
    """
    Recebe um dataframe com datas no index, ticks das ações nas colunas e volume de ações nas células
    Recebe um dataframe com retornos no mesmo formato que o descrito anteriormente
    Recebe um inteiro definindo os n maiores volumes
    
    Retorna os retornos por data de cada portfólio mensal, alocados de acordo com n maiores volumes da data 
    """
    vol_transpose = df_volume.transpose()
    
    list_lagest_volumns = [vol_transpose[row].nlargest(n_largest) for row in vol_transpose.columns]
    
    dict_largest = {list_lagest_volumns[i].name:list_lagest_volumns[i].index.values for i in range(len(list_lagest_volumns))}
    
    list_returns_ticks = []

    for k, j in dict_largest.items():
        list_returns_ticks.append(df_returns.loc[k,j])
        
    dict_returns_per_date = {str(chunck.name)[:-12]:((chunck+1).prod()-1) for chunck in list_returns_ticks}
    
    return pd.DataFrame(dict_returns_per_date, index=[0]).transpose()

In [180]:
get_portfolio_returns(volume, returns)

,0
2010-12,0.000000
2011-01,-0.448818
2011-02,0.292843
2011-03,0.246705
2011-04,-0.361151
...,...
2019-09,0.135456
2019-10,0.388448
2019-11,0.045353
2019-12,0.596044
